In [1]:
from songs import *

# ОКИ

In [2]:
from numpy import random

In [3]:
MEMORY_TIMES = 64
STARTER = 8

ADDITIONAL_MEMORY = 8

MEMORY_SIZE = MEMORY_TIMES * 13 + ADDITIONAL_MEMORY

In [4]:
NAMES = []
for i in range(8):
    NAMES.append("[tb" + str(i) + "]")
for t in range(-MEMORY_TIMES, 0):
    for note_name in ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B", "C^"]:
        NAMES.append(note_name + str(t))

In [5]:
class Func:
    def __init__(self, Random):
        self.operations = ["conj"]
        self.operands = np.zeros((1, MEMORY_SIZE))
        self.support_variables = [[]]
        self.cnt_nonzero = [0]
        self.final_decision_operation = 0
        self.Random = Random
        
    def apply(self, x):
        ans = 1
        self.final_decision_operation = -1
        
        vals = self.operands.dot(x) == self.cnt_nonzero
        
        for index, oper, val in zip(range(len(self.operations)), self.operations, vals):
            if oper == "conj" and val:
                ans = 0
                self.final_decision_operation = index
            if oper == "disj" and val:
                ans = 1
                self.final_decision_operation = index
        return ans
    
    def reward(self, memory):
        self.support_variables[self.final_decision_operation].append(frozenset(memory.nonzero()[0]))
    
    def add(self, operation, memory, where_to_put):        
        impulses = memory.nonzero()[0]
        self.Random.shuffle(impulses) 
                
        cut = 1
        problem = -1
        for i in range(where_to_put):
            if self.operations[i] != operation:
                for support_point in self.support_variables[i]:
                    while cut <= len(impulses) and set(impulses[:cut]).issubset(support_point):
                        cut += 1
                    
                    if cut > len(impulses) and set(impulses).issubset(support_point):
                        cut = len(impulses)
                        problem = i
                        break
                if problem != -1:
                    return#break
        
        '''
        if problem != -1:
            while where_to_put != problem:
                where_to_put -= 1
                
                common_set = set(self.support_variables[where_to_put][0])
                for support_point in self.support_variables[where_to_put]:
                    common_set = common_set & support_point
                
                while frozenset(self.operands[where_to_put].nonzero()[0]).issubset(set(impulses)):
                    if len(common_set) == 0:
                        print("Deadlock :(")
                        return

                    new_item = self.Random.choice(list(common_set))
                    common_set.remove(new_item)
                    self.operands[where_to_put][new_item] = 1
                    self.cnt_nonzero[where_to_put] += 1
            print("problem was solved!")
        '''
        
        mask = np.zeros((MEMORY_SIZE))
        for var in impulses[:cut]:
            mask[var] = 1            
        
        self.operations.insert(where_to_put, operation)
        self.support_variables.insert(where_to_put, [frozenset(impulses)])        
        self.operands = np.vstack([self.operands[:where_to_put], mask, self.operands[where_to_put:]])
                                  
        self.cnt_nonzero.insert(where_to_put, np.count_nonzero(mask))
    
    def print(self):
        print("0", end="")
        for oper, new_operand in zip(self.operations[1:], self.operands[1:]):
            if oper == "conj":
                print(" ∧ not", end="")
            if oper == "disj":
                print(" ∨ ", end="")
            for i in range(MEMORY_SIZE):
                if new_operand[i]:
                    print(NAMES[i], end="")
        print("")

In [6]:
class Player:
    def __init__(self):
        self.Random = np.random.RandomState(seed=179)
        self.play_note = [Func(self.Random) for i in range(13)]
        
    def play(self, memory):
        return np.array([self.play_note[i].apply(memory) for i in range(13)], dtype=int)
    
    def learn_to_play(self, song_to_learn, verbose=False):
        memory = np.zeros((MEMORY_SIZE), dtype=int)
        memory[-STARTER*13:] = song_to_learn.notes[:STARTER].flatten()

        t = STARTER
        memory[t % 8] = 1
        errors = 0

        while t < len(song_to_learn.notes):    
            output = self.play(memory)
            
            if verbose:
                print(output)

            for note in range(13):    
                if output[note] != song_to_learn.notes[t][note] or self.play_note[note].final_decision_operation == 0:
                    if song_to_learn.notes[t][note] == 1:
                        errors += 1
                        self.play_note[note].add("disj", memory, self.play_note[note].final_decision_operation + 1)
                        if verbose:
                            print("note " + str(note) + " should be pushed!")
                    if song_to_learn.notes[t][note] == 0:
                        errors += 1
                        self.play_note[note].add("conj", memory, self.play_note[note].final_decision_operation + 1)
                        if verbose:                            
                            print("note " + str(note) + " should not be pushed!")
                else:
                    self.play_note[note].reward(memory)
                

            memory = np.concatenate([np.zeros((ADDITIONAL_MEMORY)), memory[ADDITIONAL_MEMORY + 13:], song_to_learn.notes[t]])
            t += 1
            memory[t % 8] = 1
        
        if verbose:
            print("Num of Errors: ", errors)
        return errors
    
    def improvise(self, starter, length=128):
        memory = np.zeros((MEMORY_SIZE), dtype=int)
        memory[-STARTER*13:] = starter.notes[:STARTER].flatten()

        result = MySong(starter.notes[:STARTER])
        
        t = STARTER
        memory[t % 8] = 1
        while t < length:    
            output = player.play(memory)
            result.add(output)

            memory = np.concatenate([np.zeros((ADDITIONAL_MEMORY)), memory[ADDITIONAL_MEMORY + 13:], output])
            t += 1
            memory[t % 8] = 1
        result.finish()
        
        return result

## Обучение

Создаём нового игрока и грузим пока кузнечика

In [7]:
player = Player()

In [8]:
kuznechik = Song('test/track (1).mid')

Учим игрока играть кузнечика без ошибок. Ого, за одну итерацию!

In [9]:
errors = -1
while errors != 0:
    errors = player.learn_to_play(kuznechik)
    print(errors)

362
0


Проверим, что всё окей. Подадим на вход начало кузнечика, а дальше пусть играет сам (память занимается тем, что игрок сам нажимает на рояле). Поскольку так заданы правила, кузнечик будет воспроизведён точно

In [ ]:
player.improvise(kuznechik).play()

Вот как выглядит его булево правило:

In [22]:
player.play_note[4].print()

0 ∧ notG#-30 ∧ notG#-34 ∧ notC^-3 ∧ notG#-53 ∧ notE-53 ∧ notA-55 ∧ notG#-17 ∧ notE-5 ∨ E-4 ∧ notE-18 ∧ not[tb3] ∨ [tb2] ∧ notC^-2 ∧ notE-24A-22G#-6 ∧ notG#-5 ∧ notA-6 ∧ notA-3 ∧ notA-2 ∧ notA-9 ∧ notA-8 ∨ A-34 ∧ notB-54 ∧ notA-34C^-16 ∧ notB-6 ∧ notA-44A-6 ∧ notE-48A-26G#-14 ∧ not[tb4] ∧ notE-8


Окей, чтобы услышать что-то новое, нужно другое начало. Возьмём его из другой песенки

In [23]:
simple_song = Song('test/track (2).mid')
result = player.improvise(simple_song)

Первый шедевр:

In [24]:
result.play()

Всё, пора брать все песенки!

In [25]:
player = Player()

In [26]:
def addAllTransposedVersions(Songs, song):
    while song.transpose(1):
        pass

    Songs.append(copy.deepcopy(song))
    while song.transpose(-1):
        Songs.append(copy.deepcopy(song))

In [27]:
Songs = []
for i in range(1, 35):
    addAllTransposedVersions(Songs, Song('test/track (' + str(i) + ').mid'))

ERROR! out of range!
ERROR! out of range!
ERROR! out of range!


In [28]:
import random
random.shuffle(Songs)

In [ ]:
for song in Songs:
    errors = player.learn_to_play(song)
    print(errors)

In [33]:
simple_song = Song('test/whomadethis.mid')
result = player.improvise(simple_song)
result.play()

Можно подставить своё какое-нибудь начало:

In [34]:
simple_song = Song([0, -1, 5, -1, 4, -1, 5, -1])
result = player.improvise(simple_song)
result.play()

In [44]:
result.save_file("31 transposed songs greedy-conj-disj tb no zero support points [C-F-E-F]")